1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [4]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [5]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [6]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [7]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [8]:
df = df.dropna()

In [9]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [10]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [11]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [12]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [13]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'BGGId'])['Rating'].mean()

In [14]:
mean_ratings

Username        BGGId 
 Fu_Koios       112092    9.0
                223033    9.0
 beastvol       13        8.0
                118       7.0
                278       7.0
                         ... 
Æleksandr Þræð  2399      7.0
                2932      6.0
                5451      8.0
                5554      7.0
                9962      8.0
Name: Rating, Length: 18909465, dtype: float64

In [15]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [16]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df_removed_duplicates = df.merge(mean_ratings, on=['Username', 'BGGId'], suffixes=('', '_mean'))

In [17]:
df_removed_duplicates

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [18]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df_removed_duplicates[~df_removed_duplicates.duplicated(subset=['Username', 'BGGId'], keep=False)]

In [19]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [80]:
# Drop the original 'Rating' column and rename the mean rating column
df_dropped = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [81]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df_dropped = df_dropped.reindex(columns=column_order)

In [82]:
df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


Filter dataset

In [83]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
 Fu_Koios              2
 beastvol              9
 mycroft              14
 woh                   5
(mostly) harmless      1
                    ... 
zzzuzu                39
zzzvone               21
zzzxxxyyy             32
zzzzzane             154
Æleksandr Þræð        12
Length: 411312, dtype: int64

In [84]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
347146       42
347521       71
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

In [85]:
game_ratings = 3000

rating_counter = df_dropped['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df_dropped['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df_dropped.drop(index=df_dropped[games_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
849332,30549,4.0,robvw,Pandemic
849333,30549,4.0,Njaa,Pandemic
849334,30549,4.0,RussetGandalf,Pandemic
849335,30549,4.0,bigcrab,Pandemic
849336,30549,4.0,TheDS,Pandemic
...,...,...,...,...
15164074,1261,9.0,caesarmom,Medina
15164075,1261,9.0,LittleMisfit,Medina
15164076,1261,9.0,jody,Medina
15164077,1261,9.0,Lord_Indûr,Medina


In [86]:
user_ratings = 100

rating_counter = df_dropped['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df_dropped['Username'].isin(filtered_out)

df_dropped.drop(index=df_dropped[user_mask].index, inplace=True)

df_dropped

,BGGId,Rating,Username,Name
849332,30549,4.0,robvw,Pandemic
849335,30549,4.0,bigcrab,Pandemic
849344,30549,4.0,ubuntu_tyler,Pandemic
849348,30549,4.0,Kbj032,Pandemic
849363,30549,4.0,dcarlton,Pandemic
...,...,...,...,...
15164073,1261,9.0,Der Ubermolch,Medina
15164074,1261,9.0,caesarmom,Medina
15164076,1261,9.0,jody,Medina
15164077,1261,9.0,Lord_Indûr,Medina


In [87]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_dropped.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-     201
-Johnny-     245
-mIDE-       356
-toni-       137
...Hammer    123
            ... 
zzyzxuk      118
zzzabiss     296
zzzhsm       171
zzzoren      141
zzzzzane     136
Length: 31482, dtype: int64

In [88]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_dropped.groupby('BGGId').size()
grouped_counts

BGGId
1         3647
3         8711
5         9065
7         1751
10        5111
          ... 
316554    5818
317985    3028
318977    2779
328871    1826
329465    1812
Length: 1167, dtype: int64

In [89]:
df_dropped

,BGGId,Rating,Username,Name
849332,30549,4.0,robvw,Pandemic
849335,30549,4.0,bigcrab,Pandemic
849344,30549,4.0,ubuntu_tyler,Pandemic
849348,30549,4.0,Kbj032,Pandemic
849363,30549,4.0,dcarlton,Pandemic
...,...,...,...,...
15164073,1261,9.0,Der Ubermolch,Medina
15164074,1261,9.0,caesarmom,Medina
15164076,1261,9.0,jody,Medina
15164077,1261,9.0,Lord_Indûr,Medina


Train test split

In [90]:
# Define a function to split the data for each user
def user_split(data, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test

# Split the data
train_set, test_set = user_split(df_dropped, test_size=0.2)


In [91]:
train_set

,BGGId,Rating,Username,Name
0,9625,5.0,-=Yod@=-,Struggle of Empires
1,203416,8.0,-=Yod@=-,Exit: The Game – The Pharaoh's Tomb
2,175621,7.0,-=Yod@=-,Epic Card Game
3,18602,7.5,-=Yod@=-,Caylus
4,25613,7.5,-=Yod@=-,Through the Ages: A Story of Civilization
...,...,...,...,...
4754392,150376,7.0,zzzzzane,Dead of Winter: A Crossroads Game
4754393,760,7.0,zzzzzane,Battle Line
4754394,134352,7.0,zzzzzane,Two Rooms and a Boom
4754395,12,9.0,zzzzzane,Ra


In [92]:
test_set

,BGGId,Rating,Username,Name
0,198773,7.50,-=Yod@=-,Codenames: Pictures
1,77423,7.50,-=Yod@=-,The Lord of the Rings: The Card Game
2,192860,7.50,-=Yod@=-,Oceanos
3,156858,8.00,-=Yod@=-,Black Orchestra
4,217,6.75,-=Yod@=-,A la carte
...,...,...,...,...
1204355,37904,6.00,zzzzzane,Formula D
1204356,42,7.00,zzzzzane,Tigris & Euphrates
1204357,147949,7.00,zzzzzane,One Night Ultimate Werewolf
1204358,161417,7.00,zzzzzane,Red7


filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [93]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)



In [94]:
user_item_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,0.0,0.0,0.0,7.5,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
-Johnny-,0.0,0.0,7.0,0.0,4.0,6.0,0.0,7.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-mIDE-,0.0,8.0,0.0,0.0,0.0,7.5,8.0,7.0,0.0,8.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,8.0,0.0,0.0
-toni-,0.0,9.0,0.0,0.0,0.0,7.0,0.0,4.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...Hammer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzabiss,0.0,0.0,0.0,0.0,0.0,5.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzhsm,5.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
# Transpose the user-item matrix to obtain an item-user matrix
item_user_matrix = user_item_matrix.T

In [96]:
item_user_matrix

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0b1_Ita,0ddjob,0hoHj,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,0.0,0.0
3,0.0,0.0,8.0,9.0,0.0,6.0,0.0,7.0,0.0,7.9,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,5.0,7.0,0.0,8.0,0.0,0.0
7,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,10.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
317985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318977,7.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
def calculate_cosine_similarity(matrix_transposed):

    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((num_items, num_items))

    # Iterate over each row (item) in the transposed matrix by index
    #for i in range(10):
    for i in range(num_items):
        for j in range(i, num_items):  # Only compute upper triangle (similarity_matrix is symmetric)
            # Get rows by numeric position using csr_matrix row slicing
            row_i = sparse_matrix.getrow(i)
            row_j = sparse_matrix.getrow(j)

            # Find indices where both rows have non-missing values
            common_indices = np.intersect1d(row_i.indices, row_j.indices)

            if common_indices.size > 0:
                # Get non-missing values for common users
                non_missing_values_i = row_i[:, common_indices].toarray().flatten()
                non_missing_values_j = row_j[:, common_indices].toarray().flatten()
                similarity = cosine_similarity([non_missing_values_i], [non_missing_values_j])[0, 0]
                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity

    return similarity_matrix



In [98]:
# Compute cosine similarity between items
item_similarity = calculate_cosine_similarity(item_user_matrix)

In [99]:
# Create a DataFrame with item names as both rows and columns
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_similarity_df.values, 0)
item_similarity_df

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.976926,0.975871,0.965545,0.971213,0.973395,0.973338,0.967162,0.950237,0.961252,...,0.978521,0.982147,0.981325,0.977529,0.976167,0.980479,0.980834,0.980650,0.970445,0.973104
3,0.976926,0.000000,0.982467,0.974407,0.977036,0.979267,0.983613,0.974193,0.961394,0.969881,...,0.984922,0.987982,0.985237,0.987635,0.983547,0.986380,0.987059,0.984353,0.985054,0.981805
5,0.975871,0.982467,0.000000,0.971045,0.975106,0.980227,0.980302,0.976479,0.962596,0.971126,...,0.985489,0.982240,0.986519,0.985319,0.983615,0.985686,0.986527,0.984191,0.987374,0.982650
7,0.965545,0.974407,0.971045,0.000000,0.965960,0.971948,0.969809,0.964339,0.950889,0.958008,...,0.968924,0.972632,0.964162,0.976938,0.966009,0.974685,0.973997,0.973506,0.972937,0.977720
10,0.971213,0.977036,0.975106,0.965960,0.000000,0.977930,0.973560,0.972864,0.958118,0.968954,...,0.981841,0.976969,0.976649,0.980684,0.977923,0.977860,0.980692,0.980943,0.981469,0.978497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.980479,0.986380,0.985686,0.974685,0.977860,0.982499,0.985560,0.979574,0.966085,0.977122,...,0.992187,0.992620,0.992859,0.993445,0.991362,0.000000,0.993596,0.991346,0.992476,0.989894
317985,0.980834,0.987059,0.986527,0.973997,0.980692,0.985243,0.983325,0.978181,0.964608,0.976394,...,0.992757,0.992149,0.993146,0.993727,0.991507,0.993596,0.000000,0.992486,0.992321,0.985704
318977,0.980650,0.984353,0.984191,0.973506,0.980943,0.983505,0.982089,0.978271,0.972169,0.974223,...,0.991116,0.992583,0.991966,0.992444,0.989924,0.991346,0.992486,0.000000,0.992862,0.985090


In [100]:
# Convert similarity to distances (inversely proportional to similarities)
distance_matrix = 1 - item_similarity_df

In [101]:
distance_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.023074,0.024129,0.034455,0.028787,0.026605,0.026662,0.032838,0.049763,0.038748,...,0.021479,0.017853,0.018675,0.022471,0.023833,0.019521,0.019166,0.019350,0.029555,0.026896
3,0.023074,1.000000,0.017533,0.025593,0.022964,0.020733,0.016387,0.025807,0.038606,0.030119,...,0.015078,0.012018,0.014763,0.012365,0.016453,0.013620,0.012941,0.015647,0.014946,0.018195
5,0.024129,0.017533,1.000000,0.028955,0.024894,0.019773,0.019698,0.023521,0.037404,0.028874,...,0.014511,0.017760,0.013481,0.014681,0.016385,0.014314,0.013473,0.015809,0.012626,0.017350
7,0.034455,0.025593,0.028955,1.000000,0.034040,0.028052,0.030191,0.035661,0.049111,0.041992,...,0.031076,0.027368,0.035838,0.023062,0.033991,0.025315,0.026003,0.026494,0.027063,0.022280
10,0.028787,0.022964,0.024894,0.034040,1.000000,0.022070,0.026440,0.027136,0.041882,0.031046,...,0.018159,0.023031,0.023351,0.019316,0.022077,0.022140,0.019308,0.019057,0.018531,0.021503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.019521,0.013620,0.014314,0.025315,0.022140,0.017501,0.014440,0.020426,0.033915,0.022878,...,0.007813,0.007380,0.007141,0.006555,0.008638,1.000000,0.006404,0.008654,0.007524,0.010106
317985,0.019166,0.012941,0.013473,0.026003,0.019308,0.014757,0.016675,0.021819,0.035392,0.023606,...,0.007243,0.007851,0.006854,0.006273,0.008493,0.006404,1.000000,0.007514,0.007679,0.014296
318977,0.019350,0.015647,0.015809,0.026494,0.019057,0.016495,0.017911,0.021729,0.027831,0.025777,...,0.008884,0.007417,0.008034,0.007556,0.010076,0.008654,0.007514,1.000000,0.007138,0.014910


Calculate nearest neighbor

In [103]:
def find_nearest_neighbors(distance_matrix, k, BGGId):

    # Create a NearestNeighbors object
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

    # Fit the model using the item-item similarity matrix
    nearest_neighbors_model.fit(distance_matrix.values)

    # Find the index of the item
    item_index = distance_matrix.index.get_loc(BGGId)

    # Find the distance and indices of the nearest neighbors
    distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)

    return distances, neighbor_indices

Här nedan skriver vi funktionen som tar input data, user, k, top_n

5 Function RecommendItems(data, user, k, top N):
6 predicted ratings ← [];
7 for each item not in user.items do
8 N u ← set of k items similar to item that user u has rated;
9 numerator ← 0;
10 denominator ← 0;
11 for each j in N u do
12 numerator ← numerator + data[item][j] × user.items[j].rating;
13 denominator ← denominator + |data[item][j]|;
14 predicted ratings[item] ← numerator denominator ;
15 Sort predicted ratings by predicted rating in descending order;
16 recommended items ← predicted ratings;
17 recommended items ← recommended items[: top N ];
18 return recommended items

In [109]:
# def get_rated_items_for_user(username):
#     # Filter the matrix for the specified user and non-zero ratings
#     user_ratings = item_user_matrix.loc[:, username]
#     non_zero_ratings = user_ratings[user_ratings != 0]
    
#     # Extract the item IDs where the user has non-zero ratings
#     items = non_zero_ratings.index.tolist()
    
#     return items


In [151]:
def get_rated_items_for_user(username):
    # Filter the matrix for the specified user and non-zero ratings
    user_ratings = item_user_matrix.loc[:, username]
    non_zero_ratings = user_ratings[user_ratings != 0]
    
    return non_zero_ratings

In [112]:
# def get_not_rated_items_for_user(username):
#     # Filter the matrix for the specified user and non-zero ratings
#     user_ratings = item_user_matrix.loc[:, username]
#     zero_ratings = user_ratings[user_ratings == 0]
    
#     # Extract the item IDs where the user has non-zero ratings
#     items = zero_ratings.index.tolist()
    
#     return items

In [167]:
def get_not_rated_items_for_user(username):
    # Filter the matrix for the specified user and non-zero ratings
    user_ratings = item_user_matrix.loc[:, username]
    zero_ratings = user_ratings[user_ratings == 0]
    
    return zero_ratings

In [153]:
test_items_for_user = get_rated_items_for_user("-=Yod@=-")

In [154]:
test_items_for_user

BGGId
7          7.5
13         7.0
42         6.5
50         8.5
74         7.5
          ... 
271896     8.0
284083     8.0
285774     7.0
291457    10.0
318977     7.0
Name: -=Yod@=-, Length: 160, dtype: float64

In [179]:
test_not_rated_items_for_user = get_not_rated_items_for_user("-=Yod@=-")

In [180]:
test_not_rated_items_for_user

BGGId
1         0.0
3         0.0
5         0.0
10        0.0
11        0.0
         ... 
314040    0.0
316554    0.0
317985    0.0
328871    0.0
329465    0.0
Name: -=Yod@=-, Length: 1007, dtype: float64

In [134]:
# def find_rated_neighbors(neighbor_distances, neighbor_indices, rated_items):
#     neighbor_indices = neighbor_indices[0]
#     neighbor_distances = neighbor_distances[0]
#     neighbors = item_user_matrix.iloc[neighbor_indices].index.tolist()
#     rated_neighbors = []
#     rated_neighbors_distances = []

#     for i in range(len(neighbors)):
#         if neighbors[i] in rated_items:
#             rated_neighbors.append(neighbors[i])
#             rated_neighbors_distances.append(neighbor_distances[i])
    
#     return rated_neighbors_distances, rated_neighbors

In [156]:
def find_rated_neighbors(neighbor_distances, neighbor_indices, rated_items):
    neighbor_indices = neighbor_indices[0]
    neighbor_distances = neighbor_distances[0]
    neighbors = item_user_matrix.iloc[neighbor_indices].index.tolist()
    rated_neighbors = []
    rated_neighbors_distances = []

    for i in range(len(neighbors)):
        if neighbors[i] in rated_items.index:
            rated_neighbors.append(neighbors[i])
            rated_neighbors_distances.append(neighbor_distances[i])
    
    return rated_neighbors_distances, rated_neighbors

In [162]:
test_nearest_neighbors_dist, test_nearest_neighbors_ind = find_nearest_neighbors(distance_matrix, 20, 7)
test_nearest_neighbors_ind

array([[1151, 1124, 1133,  978,  766, 1153,  922, 1128,  994, 1156, 1076,
         749,  757,  879, 1144, 1129,  961, 1101,  859, 1070]])

In [163]:
test_nearest_neighbors_dist

array([[0.01151966, 0.01209342, 0.01428708, 0.01540072, 0.01590427,
        0.0161835 , 0.01670038, 0.0167337 , 0.01678018, 0.01690432,
        0.01734892, 0.01745659, 0.01754443, 0.01803929, 0.01807795,
        0.01843074, 0.01846224, 0.01855753, 0.01860134, 0.01872569]])

In [164]:
test_rated_neighbors_distances, test_rated_neighbors = find_rated_neighbors(test_nearest_neighbors_dist, test_nearest_neighbors_ind, test_items_for_user)

In [165]:
test_rated_neighbors_distances

[0.018601338193981798]

In [166]:
test_rated_neighbors

[180020]

In [170]:
test_neighbor = test_rated_neighbors[0]
test_rating_neighbor = test_items_for_user.loc[test_neighbor]

In [171]:
test_rating_neighbor

5.0

In [234]:
def recommend_items(distance_matrix, username, k, top_N):
    predicted_ratings = []
    predicted_items = []
    rated_items = get_rated_items_for_user(username)
    not_rated_items = get_not_rated_items_for_user(username)

    for item in not_rated_items.index:
        distances, neighbor_indices = find_nearest_neighbors(distance_matrix, k, item)
        rated_neighbors_distances, rated_neighbors = find_rated_neighbors(distances, neighbor_indices, rated_items)

        if len(rated_neighbors) != 0:
            numerator = 0
            denominator = 0
            for i in range(len(rated_neighbors)):
                neighbor = rated_neighbors[i]
                weight = 1 - rated_neighbors_distances[i] #transform distance into similarity
                user_rating = rated_items.loc[neighbor]
                numerator += (weight * user_rating)
                denominator += weight

            predicted_rating_item = numerator / denominator
            predicted_ratings.append(predicted_rating_item)
            predicted_items.append(item)
    
    recommended_items = pd.DataFrame(predicted_ratings, index=predicted_items, columns=['Predicted rating'])
    recommended_items.index.name = 'BGGId'
    sorted_recommendations = recommended_items.sort_values(by=['Predicted rating'], ascending=False)
    top_N_recommendations = sorted_recommendations.head(top_N)
    top_N_list = top_N_recommendations.index.tolist()

    return top_N_list, top_N_recommendations

In [228]:
# def recommend_items(distance_matrix, username, k, top_N):
#     predicted_ratings = []
#     predicted_items = []
#     rated_items = get_rated_items_for_user(username)
#     not_rated_items = get_not_rated_items_for_user(username)

#     for item in not_rated_items.index:
#         distances, neighbor_indices = find_nearest_neighbors(distance_matrix, k, item)
#         rated_neighbors_distances, rated_neighbors = find_rated_neighbors(distances, neighbor_indices, rated_items)

#         if len(rated_neighbors) != 0 and item == 1:
#             numerator = 0
#             denominator = 0
#             if len(rated_neighbors) > 1:
#                 print("item med fler än två neighbors: ", item)
#             for i in range(len(rated_neighbors)):
#                 neighbor = rated_neighbors[i]
#                 print("neighbor: ", neighbor)
#                 weight = 1 - rated_neighbors_distances[i] #transform distance into similarity
#                 print("weight: ", weight)
#                 user_rating = rated_items.loc[neighbor]
#                 print("user rating: ", user_rating)
#                 numerator += (weight * user_rating)
#                 denominator += weight

#             predicted_rating_item = numerator / denominator
#             predicted_ratings.append(predicted_rating_item)
#             predicted_items.append(item)
    
#     recommended_items = pd.DataFrame(predicted_ratings, index=predicted_items, columns=['Predicted rating'])
#     recommended_items.index.name = 'BGGId'
#     sorted_recommendations = recommended_items.sort_values(by=['Predicted rating'], ascending=False)
#     top_N_recommendations = sorted_recommendations.head(top_N)
#     top_N_list = top_N_recommendations.index.to_list


#     return top_N_list

In [235]:
test_recommended_items, test_rec_df = recommend_items(distance_matrix, "-=Yod@=-", 20, 10)
test_recommended_items

[162886, 96848, 150312, 248562, 40765, 123096, 283355, 113294, 82222, 195043]

In [236]:
test_rec_df

,Predicted rating
BGGId,
162886,10.0
96848,10.0
150312,10.0
248562,10.0
40765,10.0
123096,10.0
283355,10.0
113294,10.0
82222,10.0


In [218]:
relevant_items = test_set.loc[test_set['Rating'] >= 7]

In [219]:
relevant_items

,BGGId,Rating,Username,Name
0,198773,7.5,-=Yod@=-,Codenames: Pictures
1,77423,7.5,-=Yod@=-,The Lord of the Rings: The Card Game
2,192860,7.5,-=Yod@=-,Oceanos
3,156858,8.0,-=Yod@=-,Black Orchestra
5,66188,7.0,-=Yod@=-,Fresco
...,...,...,...,...
1204354,124742,8.0,zzzzzane,Android: Netrunner
1204356,42,7.0,zzzzzane,Tigris & Euphrates
1204357,147949,7.0,zzzzzane,One Night Ultimate Werewolf
1204358,161417,7.0,zzzzzane,Red7


In [221]:
items_in_test_set_for_user = relevant_items.loc[relevant_items['Username'] == "-=Yod@=-"]
items_in_test_set_for_user

,BGGId,Rating,Username,Name
0,198773,7.5,-=Yod@=-,Codenames: Pictures
1,77423,7.5,-=Yod@=-,The Lord of the Rings: The Card Game
2,192860,7.5,-=Yod@=-,Oceanos
3,156858,8.0,-=Yod@=-,Black Orchestra
5,66188,7.0,-=Yod@=-,Fresco
6,12333,10.0,-=Yod@=-,Twilight Struggle
7,14996,8.0,-=Yod@=-,Ticket to Ride: Europe
8,167355,8.0,-=Yod@=-,Nemesis
9,143986,7.5,-=Yod@=-,CV
11,43443,7.0,-=Yod@=-,Castle Panic


Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 